## Indicate Dynawo and Modelica package.mo

In [ ]:
#=  The env variable DYNAWO_PATH must has be set beforehands to the root of the dynawo code.
    In case you are only working with the library distribution, the second part of the expression must be slightly modified.
    Anyway the package_file should be pointing to the package.mo file of the Dynawo library you want to use
=#

Dynawo_package_file = string(ENV["DYNAWO_PATH"], "/dynawo/sources/Models/Modelica/Dynawo/package.mo")
Modelica_package_file = string(ENV["MODELICA_PATH"], "package.mo")

## Import packages and data

In [ ]:
using OMJulia
using Plots, DataFrames, CSV

In [ ]:
StaticCase = OMJulia.OMCSession()
ModelicaSystem(StaticCase,Dynawo_package_file,"Dynawo.Examples.InertialGrid.StaticCase",[Modelica_package_file])

DoubleIG = OMJulia.OMCSession()
ModelicaSystem(DoubleIG,Dynawo_package_file,"Dynawo.Examples.InertialGrid.DoubleInertialGrid",[Modelica_package_file])


## Run initial simulation

In [ ]:
buildModel(StaticCase)
simulate(StaticCase)

buildModel(DoubleIG)
resultfile_name = string("DoubleIG_base", ".csv")
simulate(DoubleIG,resultfile = resultfile_name,
                simflags = "-override=outputFormat=csv")
resultfile = joinpath(getWorkDirectory(DoubleIG), resultfile_name)
df = DataFrame(CSV.File(resultfile));

In [ ]:
plotlyjs()
p2 = plot(df[!, "time"], [df[!, "inertialGrid1.reducedOrderSFR.deltaFrequency"] df[!, "inertialGrid2.reducedOrderSFR.deltaFrequency"]], label=["deltaF_IG1" "deltaF_IG2"])
plot!(p2, legend=:bottomright, titlefontsize=12, labelfontsize=10)
title!(p2, "Frequency variations in the two inertial grids")
xlabel!(p2, "Time (s)")
ylabel!(p2, "deltaFrequency (Hz)")  

In [ ]:
plotlyjs()
p = plot(df[!, "time"], df[!, "deltaFrequency"], label=string("deltaF"))
plot!(p, legend=:bottomright, titlefontsize=12, labelfontsize=10)
title!(p, "Frequency difference between the two inertial grids")
xlabel!(p, "Time (s)")
ylabel!(p, "deltaFrequency (Hz)")

## Modification of the inertial in both grids

In [ ]:
setParameters(DoubleIG, ["inertialGrid1.H = " * "4", "inertialGrid2.H = " * "4"])
buildModel(DoubleIG)
resultfile_name = string("DoubleIG_H", ".csv")
simulate(DoubleIG,resultfile = resultfile_name,
                simflags = "-override=outputFormat=csv")
resultfile = joinpath(getWorkDirectory(DoubleIG), resultfile_name)
df = DataFrame(CSV.File(resultfile));

In [ ]:
plotlyjs()
p2 = plot(df[!, "time"], [df[!, "inertialGrid1.reducedOrderSFR.deltaFrequency"] df[!, "inertialGrid2.reducedOrderSFR.deltaFrequency"]], label=["deltaF_IG1" "deltaF_IG2"])
plot!(p2, legend=:bottomright, titlefontsize=12, labelfontsize=10)
title!(p2, "Frequency variations in the two inertial grids")
xlabel!(p2, "Time (s)")
ylabel!(p2, "deltaFrequency (Hz)")  

In [ ]:
plotlyjs()
p = plot(df[!, "time"], df[!, "deltaFrequency"], label=string("deltaF"))
plot!(p, legend=:bottomright, titlefontsize=12, labelfontsize=10)
title!(p, "Frequency difference between the two inertial grids")
xlabel!(p, "Time (s)")
ylabel!(p, "deltaFrequency (Hz)")

## Modification of the length of the lines and the variation of Pe

In [ ]:
## In case of a larger variation compared with the initial case, initial values should be modified to ease the LF convergence

setParameters(StaticCase, ["L1 = " * "150", "L2 = " * "150"])
buildModel(StaticCase)
simulate(StaticCase)
(U1, U2, U3, UPhase1, UPhase2, UPhase3, P1, Q1) = getContinuous(StaticCase, ["busIG1.UPu", "busIG2.UPu", "busL.UPu", "busIG1.UPhase", "busIG2.UPhase", "busL.UPhase", "line1.P1Pu", "line1.Q1Pu"])

setParameters(DoubleIG, ["inertialGrid1.H = " * "2.6", "inertialGrid2.H = " * "2.6"])
setParameters(DoubleIG, ["L1 = " * "150", "L2 = " * "150", "deltaPPu = " * "0.02"])
setParameters(DoubleIG, ["inertialGrid1.U0Pu = " * string(U1), "inertialGrid2.U0Pu = " * string(U2), "load.u0Pu.re = " * string(U3*cos(UPhase3)), "inertialGrid1.UPhase0 = " * string(UPhase1), "inertialGrid2.UPhase0 = " * string(UPhase2), "load.u0Pu.im = " * string(U3*sin(UPhase3)), "inertialGrid1.P0Pu = " * string(P1), "inertialGrid1.Q0Pu = " * string(Q1)])
resultfile_name = string("DoubleIG_modified", ".csv")
simulate(DoubleIG,resultfile = resultfile_name,
                simflags = "-override=outputFormat=csv")
resultfile = joinpath(getWorkDirectory(DoubleIG), resultfile_name)
df = DataFrame(CSV.File(resultfile));

In [ ]:
plotlyjs()
p2 = plot(df[!, "time"], [df[!, "inertialGrid1.reducedOrderSFR.deltaFrequency"] df[!, "inertialGrid2.reducedOrderSFR.deltaFrequency"]], label=["deltaF_IG1" "deltaF_IG2"])
plot!(p2, legend=:bottomright, titlefontsize=12, labelfontsize=10)
title!(p2, "Frequency variations in the two inertial grids")
xlabel!(p2, "Time (s)")
ylabel!(p2, "deltaFrequency (Hz)")  

In [ ]:
plotlyjs()
p = plot(df[!, "time"], df[!, "deltaFrequency"], label=string("deltaF"))
plot!(p, legend=:bottomright, titlefontsize=12, labelfontsize=10)
title!(p, "Frequency difference between the two inertial grids")
xlabel!(p, "Time (s)")
ylabel!(p, "deltaFrequency (Hz)")